In [ ]:
import requests
import os 
import zipfile
import pandas as pd
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning # 1.4.4
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter("ignore", UserWarning)
from datetime import date

In [ ]:
# Definir ulr y directorio de descarga 
ulr = "https://cdn.conahcyt.mx/enis/salud/vih/vih_red_completa-clusters_vih-notas_metodologica-tabla_relacion.zip"

directorio = "../datos/datos_originales"
nombre_archivo = "vih_descarga.zip"
directorio_guardar = os.path.join(directorio, nombre_archivo)

# Descargar archivo 
try:
    print("Descargando el archivo .zip desde la pagina web...")
    respuesta = requests.get(ulr)
    respuesta.raise_for_status()
    with open(directorio_guardar, "wb" ) as file:
        file.write(respuesta.content)
    print(f"El archivo zip fue descargado exitosamente a {directorio_guardar}")
except requests.exceptions.RequestException as e:
    print(f"Ocurrio un error durante la descarga del archivo zip: {e}")
    exit()

# Extraer archivos del zip 
try:
    print("Extrayendo el contenido del archivo zip...")
    with zipfile.ZipFile(directorio_guardar, "r") as zip_ref:
        zip_ref.extractall(directorio)
    print(f"El contenido del archivo zip fue extraido exitosamente a {directorio}") 
except zipfile.BadZipFile as e:
    print(f"Ocurrio un error durante la extracción del archivo zip: {e}")
    exit()

In [ ]:
# Lectura de bases de datos
nombre_archivo_1 = "vih_descarga/02_analisis_datos/vih_red_completa.csv"
nombre_archivo_2 = "vih_descarga/02_analisis_datos/vih_clusters.csv"
red = pd.read_csv(os.path.join('..', 'datos', 'datos_originales', nombre_archivo_1), dtype = {'id_Muestra':str}).replace({'Missing':np.nan})
clusters = pd.read_csv(os.path.join('..', 'datos', 'datos_originales', nombre_archivo_2), dtype = {'id1':str, 'id2':str})

In [ ]:
# Homologacion de fechas
red["fecha_toma"] = pd.to_datetime(red["fecha_toma"])

# Creación variables categoricas extra
red['resistencia_bool'] = red['resistencia'].replace({'Susceptible':'Sin resistencia', 'NNRTI':'Con resistencia', 
                                                        'PI':'Con resistencia','Compleja':'Con resistencia', 
                                                        'NRTI': 'Con resistencia', 'INSTI':'Con resistencia'})
red['seguridad_social_bool'] = red['seguridad_social'].replace({'IMSS': 'Con seguridad', 'ISSSTE': 'Con seguridad',
                                                                'INSABI': 'Con seguridad','Seguro popular': 'Con seguridad',
                                                                'Otro':'Con seguridad','Desconoce':'No sabe'})


rangos_distancia_genetica = pd.IntervalIndex.from_tuples([(0, 4), (4, 8), (8, 12),(12,15)],closed = 'left')
clusters.insert(0,'distancia_genetica_rango',pd.cut(clusters['distancia']*1000,rangos_distancia_genetica).replace({
       rangos_distancia_genetica[0]:"0 a 0.39", rangos_distancia_genetica[1]:"0.4 a 0.79", rangos_distancia_genetica[2]:"0.8 a 1.19",rangos_distancia_genetica[3]:"1.2 a 1.49"}))

# Agregado de fechas de toma de muestra 
red_con_clusters = list(set(clusters['id1']).union(set(clusters['id2'])))
fechas_toma_muestra = red[red['id_muestra'].isin(red_con_clusters)][['id_muestra','fecha_toma']].set_index('id_muestra')

dic_fechas = fechas_toma_muestra.to_dict()['fecha_toma']
clusters.insert(0,'fecha_toma_id1',clusters['id1'].replace(dic_fechas))
clusters.insert(0,'fecha_toma_id2',clusters['id2'].replace(dic_fechas))

clusters['distancia'] = clusters['distancia'].apply(lambda x: x*100)

In [ ]:
# Exportar bases de datos que se utilizaran en cada notebook

# ESelección de columnas de interes 
lcols = ['id1', 'id2', 'fecha_toma_id1', 'fecha_toma_id2', 'distancia', 'grado','distancia_genetica_rango']

# Datos para vizualizaciones 
clusters = clusters[lcols].set_index('id1').to_csv(os.path.join('..', 'datos', 'datos_procesados', 'enlaces'+'.csv'))
red.set_index('id_muestra').to_csv(os.path.join('..', 'datos', 'datos_procesados', 'nodos.csv'))

# Datos con información geográfica
red = red.rename({'cv':'carga_viral','cd4_a':'linfocitos'}, axis = 'columns')
red['resistencia'] = red['resistencia'].replace({'Sin resistencia':'Susceptible'})
cols_geoserver = ['id_muestra','cp','fecha_toma','estado','municipio','sexo','edad','seguridad_social','resistencia','linfocitos','carga_viral']
red[cols_geoserver].set_index('id_muestra').to_csv(os.path.join('..', 'datos', 'datos_procesados', 'sig_nodos.csv'))